In [1]:
! pip install ultralytics Roboflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 914.7/914.7 kB 13.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.0/83.0 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [2]:
from ultralytics import YOLO
from roboflow import Roboflow

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [7]:
rf = Roboflow(api_key="hiXB6MTQOH0hlxROddmK")
project = rf.workspace("jeevan-sendur-g-workspace").project("act-prediction-kmshx")
version = project.version(2)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to act-prediction-2 in yolov8:: 100%|██████████| 848/848 [00:00<00:00, 7737.15it/s]


In [4]:
import cv2
import os

# Define the input folder (where images are located)
input_folder = "/content/act-prediction-2/train/images"

# Ensure the input folder exists
if not os.path.exists(input_folder):
    print("Input folder does not exist!")
    exit()

# Process each image in the folder
for filename in os.listdir(input_folder):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):  # Only process image files
        file_path = os.path.join(input_folder, filename)

        # Load the image
        image = cv2.imread(file_path)

        if image is None:
            print(f"Skipping {filename} (could not read)")
            continue

        # apply Median Blur (good for salt-and-pepper noise)
        median_blur = cv2.medianBlur(image, 5)
        # resize to 640 x 640
        resized = cv2.resize(median_blur, (640, 640))
        # grayscale
        final = cv2.cvtColor(resized, cv2.COLOR_BGR2GRAY)

        # Save back with the same filename
        output_path = os.path.join(input_folder, filename)
        cv2.imwrite(output_path, final)

print("preprocessing complete!")


Noise reduction processing complete!


In [ ]:
# the above preprocessing block is not really necessary, preprocessing and augmentations already done via roboflow

In [5]:
model = YOLO("yolov8x.pt")

100%|██████████| 131M/131M [00:00<00:00, 311MB/s]


In [9]:
results = model.train(data="/content/act-prediction-2/data.yaml", epochs=50, imgsz=640, batch=0.9) #batch=0.9 means batchsize is dynamically chosen to utilize upto a maximum of 90% of GPU memory

Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolov8x.pt, data=/content/act-prediction-2/data.yaml, epochs=50, time=None, patience=100, batch=0.9, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train22, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

train: Scanning /content/act-prediction-2/train/labels.cache... 388 images, 0 backgrounds, 0 corrupt: 100%|██████████| 388/388 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
AutoBatch: Computing optimal batch size for imgsz=640 at 90.0% CUDA memory utilization.
AutoBatch: CUDA:0 (Tesla T4) 14.74G total, 3.01G reserved, 1.83G allocated, 9.90G free
      Params      GFLOPs  GPU_mem (GB)  forward (ms) backward (ms)                   input                  output


    68156460       258.1         3.615         49.76         70.37        (1, 3, 640, 640)                    list
    68156460       516.3         4.849         63.38         96.25        (2, 3, 640, 640)                    list
    68156460        1033         6.730         99.72         170.9        (4, 3, 640, 640)                    list
    68156460        2065        10.316           190         319.3        (8, 3, 640, 640)                    list
    68156460        4130        16.553         370.7         628.5       (16, 3, 640, 640)                    list
AutoBatch: Using batch-size 6 for CUDA:0 13.45G/14.74G (91%) ✅


train: Scanning /content/act-prediction-2/train/labels.cache... 388 images, 0 backgrounds, 0 corrupt: 100%|██████████| 388/388 [00:00<?, ?it/s]

...

      50/50      6.09G     0.2652     0.2304     0.8772          6        640: 100%|██████████| 65/65 [00:33<00:00,  1.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.92it/s]

                   all         15         28      0.879      0.859       0.93      0.624



50 epochs completed in 0.660 hours.
Optimizer stripped from runs/detect/train22/weights/last.pt, 136.7MB
Optimizer stripped from runs/detect/train22/weights/best.pt, 136.7MB

Validating runs/detect/train22/weights/best.pt...
Ultralytics 8.3.74 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 268 layers, 68,127,420 parameters, 0 gradients, 257.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:00<00:00,  3.64it/s]


                   all         15         28      0.911      0.889      0.939      0.655
                Danger         11         11      0.798      0.721       0.85      0.559
              Innocent          6          6          1      0.833      0.917      0.893
             need help          9          9          1          1      0.995      0.978
                weapon          2          2      0.845          1      0.995      0.189
Speed: 0.5ms preprocess, 28.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train22


In [ ]:
# test on video
model = YOLO("yolo11n.pt")

source = "anomalousactivity.mp4"

results = model(source, stream=True)

In [ ]:
# test on live camera
results = model(source=0, stream=True)

## Precision = 91.1%
## Recall = 88.9%
## mAP = 93.9%